# 1. オペレーティングシステム 演習 14
#  mmapの課題

名前と学生証番号を書け. Enter your name and student ID.

 * 名前 Name:
 * 学生証番号 Student ID:

# 2. mmapを利用した高速な検索
* 整列された配列の2分探索をするbsearchというライブラリ関数がある(man bsearch参照)
* 以下は使用例 (配列aが昇順に整列されていることに注意)

In [ ]:
%%writefile bsearch.c
#include <stdio.h>
#include <stdlib.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int key = (argc > 1 ? atoi(argv[1]) : 100);
  int a[] = { 100, 200, 300, 400, 500 };
  int n = sizeof(a) / sizeof(a[0]);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  return 0;
}

In [ ]:
gcc -o bsearch -O3 -Wall bsearch.c

* 実行例

In [ ]:
./bsearch 100
./bsearch 200
./bsearch 400
./bsearch 390

* 以下で, 多数の32 bit整数が隙間なく, 128M個, 昇順に整列されたファイルを作る (512MB)

In [ ]:
#!/usr/bin/python3
import os
import sys
import getpass
import numpy as np

def mk_sorted_file(filename, size):
    assert(size <= 128 * 1024 * 1024), "don't make it > 512MB"
    rg = np.random.RandomState()
    rg.seed(1234)
    gap = 1024 * 1024 * 1024 / size
    a = rg.randint(0, gap, size=size, dtype=np.uint32)
    s = 0
    for i in range(size):
        s += a[i]
        a[i] = s
    dirname = os.path.dirname(filename)
    if dirname == "":
        dirname = "."
    os.makedirs(dirname, exist_ok=True)
    with open(filename, "wb") as wp:
        a.tofile(wp)

def main():
    filename = sys.argv[1]
    sz = int(sys.argv[2])
    mk_sorted_file(filename, sz)
        
if sys.argv[0].endswith("mk_sorted_file.py"):
    main()


* 多少(20秒くらい)かかるが気長に待つ

In [ ]:
n = 128 * 1024 * 1024
mk_sorted_file("sorted.bin", n)

* 確認

In [ ]:
ls -l sorted.bin

# <font color="green"> Problem 1 :  mmapの有効な利用</font>

* ファイル名と検索したい整数Kを与えられ, その整数がファイル中にあれば,
```
193659 found at 25850-th element
```
のように, その数(193659)がファイル中の何番目(最初の数を0-th elementとする)であるかを表示し, なければ,
```
19365 not found
```
のように, ないと答えるプログラムを, mmapを有効に用いて書け
* マニュアルページなどを参照しながら必要な ヘッダファイルを #include して, 一から書いてみよ

In [ ]:

%%writefile mmap_bsearch.c

int main() {



  return 0;
}

In [ ]:
gcc -o mmap_bsearch -Wall mmap_bsearch.c

* テスト

In [ ]:
data=sorted.bin
if /usr/bin/time ./mmap_bsearch ${data} 26127088  | grep "26127088 found at 7466042-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 173640831 | grep "173640831 found at 49610854-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 33502494  | grep "33502494 found at 9572459-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 413005234 | grep "413005234 found at 118006295-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 261784903 | grep "261784903 found at 74798397-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 88660114  | grep "88660114 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 347238073 | grep "347238073 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 417070716 | grep "417070716 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 389200408 | grep "389200408 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 225789696 | grep "225789696 not found" ; then echo OK ; else echo NG ; fi

# <font color="green"> Problem 2 :  readとの比較</font>

同じ仕様のプログラムを, 以下の二つの方法でも実装せよ

1. read関数でファイルを大きな配列に全て読み込み, bsearchを使う

In [ ]:

%%writefile read_bsearch.c

int main() {



  return 0;
}

In [ ]:
gcc -o read_bsearch -Wall -O3 read_bsearch.c

In [ ]:
data=sorted.bin
if /usr/bin/time ./read_bsearch ${data} 26127088  | grep "26127088 found at 7466042-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 173640831 | grep "173640831 found at 49610854-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 33502494  | grep "33502494 found at 9572459-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 413005234 | grep "413005234 found at 118006295-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 261784903 | grep "261784903 found at 74798397-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 88660114  | grep "88660114 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 347238073 | grep "347238073 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 417070716 | grep "417070716 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 389200408 | grep "389200408 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 225789696 | grep "225789696 not found" ; then echo OK ; else echo NG ; fi

2\. read関数でファイルを先頭から一定数読んでは, 与えられた値と比較する

In [ ]:

%%writefile linear_search.c

int main() {



  return 0;
}

In [ ]:
gcc -o linear_search -Wall -O3 linear_search.c

In [ ]:
data=sorted.bin
if /usr/bin/time ./linear_search ${data} 26127088  | grep "26127088 found at 7466042-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 173640831 | grep "173640831 found at 49610854-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 33502494  | grep "33502494 found at 9572459-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 413005234 | grep "413005234 found at 118006295-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 261784903 | grep "261784903 found at 74798397-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 88660114  | grep "88660114 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 347238073 | grep "347238073 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 417070716 | grep "417070716 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 389200408 | grep "389200408 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./linear_search ${data} 225789696 | grep "225789696 not found" ; then echo OK ; else echo NG ; fi